In [1]:
# Imports needed for this ipython file

from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
import pickle
from TwitterAPI import TwitterAPI

%matplotlib inline

In [2]:
def get_twitter(config_file):
    """ 
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [3]:
def robust_request(twitter, resource, params, max_tries=2):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)    
        if request.status_code == 200:
            return request
        elif "Not authorized" in request.text:
            return -1
        elif "page does not exist" in request.text:
            return -2
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)

In [4]:
def get_followers(screen_name):
    """ Return a dictionary of the users that follows this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of dictionary, one per follower containing {index, user_info} pairs.
    Note: Many users follow more than 200 accounts; we will return those accounts too.
    """
    followers = {}
    cursor = -1
    key = 0
    rate_limit = 1
    while True:
        if rate_limit == 15:
            rate_limit = 1
            print >> sys.stderr, 'Avoided rate_limit error: \nsleeping for 15 minutes.'
            time.sleep(61 * 15)
        request = robust_request(twitter,'followers/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        json_response = request.json()
        
        # Get 200 followers from json_response
        for follower in json_response['users']:
                followers[follower['screen_name']] = follower
                key = key + 1
                
        # If user has < 200 followers or to retrieve last 200 followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 followers
        cursor = json_response["next_cursor"]
        rate_limit = rate_limit + 1
        print "Collected %d followers" % key
    return followers

In [5]:
def get_followers_and_pickle_object(account_name):
    """ Calls get_followers() to collect all followers of a Twitter account
        and then saves it by pickling for future processing purposes.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    followers = get_followers(account_name)
    fileObject = open("raw_data/" + account_name,'wb')
    pickle.dump(followers,fileObject)
    fileObject.close()
    return followers

In [6]:
def get_followers_from_pickle(account_name):
    """ Returns all followers of a Twitter account by retrieving saved data,
        using pickle method.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    fileObject = open("raw_data/" + account_name,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    return followers

In [61]:
# Getting iit alumni association followers and pickling them
iit_alumni_followers = get_followers_and_pickle_object('iitalumniassn')

In [7]:
iit_alumni_followers = get_followers_from_pickle('iitalumniassn')
print "Found %d followers for iitalumniassn" %len(iit_alumni_followers.keys())

Found 1861 followers for iitalumniassn


In [6]:
# Getting iit career services followers and pickling them
iit_career_followers = get_followers_and_pickle_object('IITCareers')

In [8]:
iit_career_followers = get_followers_from_pickle('IITCareers')
print "Found %d followers for IITCareers" %len(iit_career_followers.keys())

Found 643 followers for IITCareers


In [ ]:
# Getting iit main university followers and pickling them
iit_univ_followers = get_followers_and_pickle_object('illinoistech')

In [9]:
iit_univ_followers = get_followers_from_pickle('illinoistech')
print "Found %d followers for IIT University" %len(iit_univ_followers.keys())

Found 5426 followers for IIT University


In [ ]:
# Getting uiuc main university followers and pickling them
uiuc_univ_followers = get_followers_and_pickle_object('Illinois_Alma')

In [16]:
uiuc_univ_followers = get_followers_from_pickle('Illinois_Alma')
print "Found %d followers for UIUC University" %len(uiuc_univ_followers.keys())

Found 40017 followers for UIUC University


In [ ]:
# Getting uiuc alumni followers and pickling them
uiuc_alumni_followers = get_followers_and_pickle_object('UIAA')

In [45]:
uiuc_alumni_followers = get_followers_from_pickle('UIAA')
print "Found %d followers for UIUC Alumni" %len(uiuc_alumni_followers.keys())

Found 5960 followers for UIUC Alumni


In [8]:
def get_all_screen_names(*args):
    """ Returns union of all followers of a particular university
    
    Args: Variables containing followers from different accounts in dictionary format.
    
    Returns : A set of intersection of keys from the dictionaries passed as input.
    """
    univ_followers = set()
    for followers in args:
        for user in followers.keys():
            univ_followers.add(user)
    return univ_followers

In [11]:
iit_followers = get_all_screen_names(iit_alumni_followers,iit_career_followers,iit_univ_followers)
print "Found %d unique accounts from %d accounts " %(len(iit_followers), 
            (len(iit_alumni_followers)+len(iit_career_followers)+len(iit_univ_followers)))

Found 6565 unique accounts from 7930 accounts 


In [13]:
# Pickling iit_followers for future calculations.
fileObject = open("processed_data/iit_followers",'wb')
pickle.dump(iit_followers,fileObject)
fileObject.close()

In [7]:
# Loading iit_followers from pickle.
fileObject = open("processed_data/iit_followers",'r')
iit_followers = pickle.load(fileObject)
fileObject.close()

In [10]:
def get_friends(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A dictionary of {screen_name : description} per friend of the given account.
    """
    friends = {}
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/list', 
                                 {'screen_name': screen_name, 'count':200, 'cursor': cursor})
        
        if request == None:
            return 0
        
        if request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:
            return -2
        
        json_response = request.json()
        
        # Get 200 friends from json_response
        for friend in json_response['users']:
                #friends.append(friend['screen_name']) -- to_remove
                friends[friend['screen_name']] = friend['description']
                key = key + 1
                
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [ ]:
friends_of_iit_followers = { x : get_friends(x) for x in iit_followers}

In [11]:
def get_friends_and_pickle_object(univ_followers, univ_name):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friends(follower)
        friends_of_univ_followers[follower] = friends
        fileObject = open("raw_data/friends_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
    return friends_of_univ_followers

In [ ]:
get_friends_and_pickle_object(iit_followers,"iit")

In [9]:
for idx,follower in enumerate(iit_followers):
    if "hardik_setia" in follower:
        print idx

2777


In [21]:
iit_followers_2 =  list(iit_followers)[1548:]

In [14]:
def get_friend_ids(screen_name):
    """ Return a dictionary of users that are friends of this person on Twitter.
    
    Args:
        screen_name: a string of a Twitter screen name
    Returns:
        A list of ids per friend of the given account.
    """
    friends = []
    cursor = -1
    key = 0
    while True:
        request = robust_request(twitter,'friends/ids', 
                                 {'screen_name': screen_name, 'count':5000, 'cursor': cursor})
        
        if request == None:
            return 0
        elif request == -1 :    # Authentication error : Can't access follower data
            return -1
        elif request == -2:   # Page does not exist error
            return -2
        
        json_response = request.json()
        
        # Get 5000 friends from json_response
        friends = json_response['ids']
        key = len(friends)
                
        # If user has < 200 friends or to retrieve last friends followers
        if json_response["next_cursor"] == 0:
            break
            
        # Update cursor to get next 200 friends
        cursor = json_response["next_cursor"]
        print "Collected %d friends for %s" % (key,screen_name)
    print "Returning --- %d friends for %s" % (key,screen_name)
    return friends

In [13]:
def get_friend_ids_and_pickle_object(univ_followers, univ_name):
    """ Calls get_friends() to collect all friends of a Twitter account
        and then saves it by pickling.
    
    Args: A list of strings representing the screen_names of all followers of a university,
          A string representing the university's name.
          
    Returns : A dictionary of { String , [List] } pairs,
              where String is a university follower's screen_name
              and List is a list of screen_names of their friends
    """
    friends_of_univ_followers = {}
    for follower in univ_followers:
        friends = get_friend_ids(follower)
        friends_of_univ_followers[follower] = friends
        fileObject = open("raw_data/friend_ids_of_" + univ_name + "_followers",'wb')
        pickle.dump(friends_of_univ_followers,fileObject)
        fileObject.close()
    return friends_of_univ_followers

In [ ]:
get_friend_ids_and_pickle_object(iit_followers_2,"iit2")

Returning --- 2511 friends for DinkyShop
Returning --- 1985 friends for FundasMadeEasy
Returning --- 1788 friends for BellevueU
Returning --- 28 friends for SUSHANTKUMARS14
Returning --- 1994 friends for StopVirtuaStalk
Returning --- 277 friends for akp4595
Returning --- 138 friends for miyfghiuwdfd
Returning --- 283 friends for gkdelhi8
Returning --- 34 friends for onlyAbhiJ
Returning --- 20 friends for yogeshjshah
Returning --- 51 friends for Twu28Wu
Returning --- 361 friends for SerialParoxysm
Returning --- 13 friends for Sumesh_Sethu

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 2894 friends for PearsonSci
Returning --- 20 friends for muralim97510723
Returning --- 4 friends for pxportill
Returning --- 141 friends for nikki_does
Returning --- 53 friends for yadavAshish11
Returning --- 1962 friends for liethyderabad
Returning --- 30 friends for iitsjm
Returning --- 430 friends for jrnag2
Returning --- 155 friends for vas_ashwin
Returning --- 237 friends for cheesybrownie
Returning --- 787 friends for SCTransferCtr
Returning --- 8 friends for raer77
Returning --- 47 friends for meetsharmitha
Returning --- 11 friends for pavansl

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 33 friends for SklmTown
Returning --- 139 friends for ravikalsia
Returning --- 839 friends for ED24group
Returning --- 232 friends for klillehaugCPVA
Returning --- 105 friends for PaigeGrons
Returning --- 398 friends for Naveenprasant12
Returning --- 1562 friends for jscotheathman
Returning --- 70 friends for OmNarkar
Returning --- 34 friends for iititm
Returning --- 672 friends for nicolenetland
Returning --- 30 friends for SingleHopKyle
Returning --- 930 friends for CFLCruises
Returning --- 1529 friends for dynegy
Returning --- 159 friends for IITFaculty

...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 54 friends for AnkithMJain
Returning --- 43 friends for jasondtd
Returning --- 221 friends for Jason_Minnich
Returning --- 155 friends for WareIsRachel
Returning --- 165 friends for wooddragonus
Returning --- 1971 friends for KneeShit
Returning --- 113 friends for scorpiknox
Returning --- 527 friends for TheTradingShow
Returning --- 99 friends for iitarc
Returning --- 109 friends for sandra_casten
Returning --- 651 friends for pure_oso
Returning --- 142 friends for jrwall9
Returning --- 1284 friends for nayanrahul08

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 392 friends for HaridassSns
Returning --- 209 friends for Bhakthavatsala
Returning --- 58 friends for sudhirksingh27
Returning --- 815 friends for spicymon
Returning --- 161 friends for Comillas_SRI
Returning --- 1677 friends for GGIPanipat
Returning --- 19 friends for nikhilsakhare3
Returning --- 157 friends for brooks_1
Returning --- 297 friends for juanchi_hdez
Returning --- 39 friends for SerkanTasci
Returning --- 1222 friends for MaximusMarq
Returning --- 2703 friends for ZipcarChicago
Returning --- 85 friends for hardik_setia
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


In [11]:
iit_followers_3 =  list(iit_followers)[2777:]
print len(iit_followers_3)

3788


In [15]:
get_friend_ids_and_pickle_object(iit_followers_3,"iit3")

Returning --- 85 friends for hardik_setia
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Collected 5000 friends for CrainsChicago
Returning --- 3195 friends for CrainsChicago
Returning --- 160 friends for ashraf5486
Returning --- 97 friends for inayemm
Returning --- 92 friends for BogIIT
Returning --- 73 friends for tsm96tusshar
Returning --- 377 friends for GreenBeautyCEO

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 209 friends for billzhangsh
Returning --- 128 friends for MerchMartEvents
Collected 5000 friends for summerlearning
Returning --- 345 friends for summerlearning
Returning --- 502 friends for RRJay_
Returning --- 1960 friends for Hace_Online
Returning --- 806 friends for LearningLateral
Returning --- 505 friends for whystudyhere
Returning --- 15 friends for joy93_joy
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Collected 5000 friends for CrowdTMeeting
Returning --- 716 friends for CrowdTMeeting

...

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Collected 5000 friends for bernardmoon
Collected 5000 friends for bernardmoon
Returning --- 3396 friends for bernardmoon
Returning --- 1141 friends for Retail_Careers
Returning --- 120 friends for jasduke_
Returning --- 9 friends for byumilove
Returning --- 481 friends for HarrysFrozenD
Returning --- 110 friends for DBEducation
Returning --- 1212 friends for bhaggu
Returning --- 19 friends for aaron1025xq
Returning --- 270 friends for onkar_11
Returning --- 1984 friends for CPExpo
Returning --- 1157 friends for CRZZRS

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



Returning --- 696 friends for pridkett
Returning --- 13 friends for santhooravi
Returning --- 41 friends for SeanZhang2012
Returning --- 94 friends for nancyzhang2
Returning --- 1997 friends for cybercrimemanua

TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)